In [1]:
from pywaterinfo import Waterinfo
import os
import pandas as pd
import numpy as np
import schedule
import datetime
import time

In [2]:
df_last_hour= None
df_year = None
vmm = Waterinfo("vmm") #flanders enviroment agency
hic = Waterinfo("hic") #flanders hydraulics research
outputfile = 'data.csv'
set_year = None
set_hour = None

In [ ]:
hic.get_group_list().to_csv('hic_list.csv',index=False,mode='w')
vmm.get_group_list().to_csv('vmm_list.csv',index=False,mode='w')

In [24]:
def fetch_year():
    print("fetching data from last year")
    global df_year
    df_year = vmm.get_timeseries_values(78124042, period="P5Y")
    #vmm.get_timeseries_values(78124042,start="2016-01-01",end="2016-01-02")
    print("last year data fetched successfully")

In [4]:
def fetch_hourly():
    print("fetching data from last hour")
    global df_last_hour
    df_last_hour = vmm.get_timeseries_values(78124042, period="PT1H")
    print("last hour data fetched successfully")

In [5]:
def combine():
    print("data is being combined")
    global set_year, set_hour
    if df_year is None or df_last_hour is None:
        print("Error: One or both DataFrames are empty.")
        return
    # Union of both sets (non-duplicates)
    set_year = set([tuple(x) for x in df_year.values])
    set_hour = set([tuple(x) for x in df_last_hour.values])
    set_year = set_year.union(set_hour)
    print("data combined successfully")

In [6]:
def initial_write():
    print("file initialization started")
    if df_year is  None:
        print("Error: DataFrame is empty.")
        return
    df_year.to_csv('data.csv',index=False,mode='w')
    print("file initialization successful")

In [7]:
def write_file():
    print("file writing started")
    global set_year ,df_year
    if set_year is None or df_year is None:
        print("Error: No data to write.")
        return
    # Convert back to a DataFrame
    df_unique = pd.DataFrame(list(set_year), columns=df_year.columns)
    df_unique = df_unique.sort_values(by='Timestamp')
    df_unique.to_csv('data.csv',index=False,mode='w')
    print("file writing finished: ",datetime.datetime.now())


In [25]:
def main():
    print("program started: ",datetime.datetime.now())
    fetch_year()
    initial_write()
    schedule.every(30).minutes.do(fetch_hourly)
    schedule.every(30).minutes.do(combine)
    schedule.every(12).hours.do(write_file) #this will later be changed to 24h
    print("Main function is running, tasks are scheduled.")
    # Run the schedule loop
    while True:
        schedule.run_pending()  # Run the scheduled tasks
        time.sleep(1) 


# Check if the script is being run directly (not imported as a module)
if __name__ == "__main__":
    main()

program started:  2025-03-06 12:50:33.000530
fetching data from last year
last year data fetched successfully
file initialization started
file initialization successful
Main function is running, tasks are scheduled.
fetching data from last hour
last hour data fetched successfully
data is being combined
data combined successfully
fetching data from last hour
last hour data fetched successfully
data is being combined
data combined successfully


KeyboardInterrupt: 

# Experiments

In [12]:
#check if hic and vmm have the same dimentions
df_year = vmm.get_timeseries_values(78124042, period="P1Y")


In [13]:
vmm.get_group_list()

,group_id,group_name,group_type
0,11512,Webinput,station
1,85063,Toestandsrapport stations,station
2,191093,Droogtestations,station
3,195125,ZZZ_Webinput_DL_stations,station
4,293754,Wachtbekkens,station
...,...,...,...
427,456310,FEWSDynamisch_HMeetperiodenPieken,timeseries_dynamic
428,458913,WISKIDynamisch_VMM_IOT_IJZER,timeseries_dynamic
429,368710,H_Wachtbekkens_samenwerkingsposten,parameter
430,409463,tmp_waterpeil-parameters meer dan 1 jaar,parameter


In [29]:
vmm.get_timeseries_value_layer(timeseriesgroup_id="456310")

,ts_id,timestamp,req_timestamp,ts_value,station_latitude,station_longitude,ts_name,ts_shortname,station_no,station_id,station_name,stationparameter_name,stationparameter_no,stationparameter_longname,ts_unitname,ts_unitsymbol,parametertype_id,parametertype_name,ts_path,dataprovider
0,314703042,2025-02-28T04:15:00.000Z,None,4.714,51.146806,3.831139,MeetperiodenPieken,Cmd.LTV.POR.POT.Abs,L03_301,15494,Mendonk/Zuidlede,H,H,River Stage,meter,m,559,H,AOW_LIMNIGRAFEN/L03_301/H/Cmd.LTV.POR.POT.Abs,VMM
1,157278042,2025-02-11T14:00:00.000Z,None,34.482,50.768578,4.604977,MeetperiodenPieken,Cmd.LTV.POR.POT.Abs,L08_118,12101,Overijse/Laan,H,H,River Stage,meter,m,559,H,AOW_LIMNIGRAFEN/L08_118/H/Cmd.LTV.POR.POT.Abs,VMM
2,314724042,None,None,NaN,51.019378,4.534930,MeetperiodenPieken,Cmd.LTV.POR.POT.Abs,L08_11U,429461,Bonheiden/Vrouwvliet/,H,H,River Stage,meter,m,559,H,AOW_LIMNIGRAFEN/L08_11U/H/Cmd.LTV.POR.POT.Abs,VMM
3,157939042,2025-01-27T17:45:00.000Z,None,16.251,51.077777,4.986346,MeetperiodenPieken,Cmd.LTV.POR.POT.Abs,L10_085,12157,Veerle/Grote Laak,H,H,River Stage,meter,m,559,H,AOW_LIMNIGRAFEN/L10_085/H/Cmd.LTV.POR.POT.Abs,VMM
4,156574042,2025-02-11T12:30:00.000Z,None,22.762,50.954569,4.234859,MeetperiodenPieken,Cmd.LTV.POR.POT.Abs,L04_038,12064,Merchtem/Grote Molenbeek,H,H,River Stage,meter,m,559,H,AOW_LIMNIGRAFEN/L04_038/H/Cmd.LTV.POR.POT.Abs,VMM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,281674042,2025-01-25T23:30:00.000Z,None,75.864,50.811871,5.377722,MeetperiodenPieken,Cmd.LTV.POR.POT.Abs,L09_16C,395872,Kerniel/Sint-Annabeek,H,H,River Stage,meter,m,559,H,AOW_LIMNIGRAFEN/L09_16C/H/Cmd.LTV.POR.POT.Abs,VMM
208,314677042,2025-02-11T20:15:00.000Z,None,22.110,50.957920,5.116455,MeetperiodenPieken,Cmd.LTV.POR.POT.Abs,L09_151,12125,Halen/Gete+Herk,H,H,River Stage,meter,m,559,H,AOW_LIMNIGRAFEN/L09_151/H/Cmd.LTV.POR.POT.Abs,VMM
209,156667042,2025-02-26T11:00:00.000Z,None,19.534,51.381253,4.827751,MeetperiodenPieken,Cmd.LTV.POR.POT.Abs,L11_048,12173,Merksplas/Mark,H,H,River Stage,meter,m,559,H,AOW_LIMNIGRAFEN/L11_048/H/Cmd.LTV.POR.POT.Abs,VMM
210,156731042,2024-05-17T22:45:00.000Z,None,61.542,50.756746,5.713314,MeetperiodenPieken,Cmd.LTV.POR.POT.Abs,L11_561,12182,Moelingen/Berwijn,H,H,River Stage,meter,m,559,H,AOW_LIMNIGRAFEN/L11_561/H/Cmd.LTV.POR.POT.Abs,VMM
